# Artificial Neural Networks (supervised)
In this section, we will train an Artificial Neural network with the same input as we trained the KNN algorithm. We start by generating the signals and the database.

In [39]:
import numpy as np
import numpy.typing as npt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from numpy.random import MT19937, RandomState, SeedSequence
from ipywidgets import widgets
from IPython.display import display, Audio
import matplotlib.pyplot as plt

random_state = 123456789
rs = RandomState(MT19937(SeedSequence(random_state)))
ws = {'description_width': 'initial'}
sample_rate = widgets.IntSlider(40000, min=10000, max=50000, description="Sample frequency (hZ)", style=ws)
display(sample_rate)

IntSlider(value=40000, description='Sample frequency (hZ)', max=50000, min=10000, style=SliderStyle(descriptio…

In [40]:
def generate_noisy_signal(
    sample_frequency: int,
    amplitudes: npt.NDArray[np.float64],
    frequencies: npt.NDArray[np.float64],
    phases: npt.NDArray[np.float64], noise: float=1, T:float=2.):
    N = int(T*sample_frequency)
    t = np.linspace(0, T, N, endpoint=False)
    assert len(frequencies) == len(phases)
    assert len(amplitudes) == len(phases)
    n = noise*np.random.randn(len(t))
    return t, n+sum([A*np.cos(2*np.pi*f*t+p) for (A,f,p) in zip(amplitudes, frequencies, phases)])

In [45]:
def generate_database(num_samples:int, T:float, num_windows:int):
    A = rs.uniform(-4, 4, size=num_samples)
    phase = np.pi * rs.random(num_samples)
    noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)
    frequencies = rs.uniform(0, 20000, size=num_samples) 
    features = np.zeros((num_samples, num_windows))
    labels = np.zeros(num_samples, dtype=np.int32)
    for i in range(num_samples):
        _, signal = generate_noisy_signal(sample_rate.value, A[i:i+1], frequencies[i:i+1],
                                          phase[i:i+1],noise[i:i+1], T)
        features[i] = extract_features(signal, num_windows, sample_rate.value)
        labels[i] = 1 + np.int32(frequencies[i] > 8e3)
    target_map = {0:"low", 1:"high"}
    feature_names = [f"Signal amplitude integrated in frequency window {i}" for i in range(num_windows)]

    return features, labels, feature_names, target_map

def extract_features(signal:npt.NDArray[np.float64], N:int, sampling_frequency:int):
    rfft = np.fft.rfft(signal)
    f_per_band = len(rfft)//N
    rem = len(rfft) % N
    freq_per_band = np.full(N, f_per_band, dtype=np.int32)
    freq_per_band[:rem]+=1
    offsets= np.zeros(N+1, dtype=np.int32)
    offsets[1:] = np.cumsum(freq_per_band)
    RMS_amplitude = np.zeros(N, dtype=np.float64)
    mag = 1/len(signal) * np.abs(rfft)
    for i in range(N):
        RMS_amplitude[i] = np.sqrt(1/(offsets[i+1]-offsets[i])*np.sum(mag[offsets[i]:offsets[i+1]]**2))
    return RMS_amplitude

T = 0.1
num_samples = 3000
num_windows = 25
features, labels, feature_names, target_map = generate_database(T=T, num_samples=num_samples, num_windows=num_windows)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=random_state)


/tmp/ipykernel_49995/3158994771.py:4: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)


In [46]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier

@widgets.interact(
    A=widgets.FloatSlider(2, min=0, max=4, description="Amplitude of wave 1", style=ws),
    B=widgets.FloatSlider(3, min=0, max=4, description="Amplitude of wave 2", style=ws),
    fA=widgets.IntSlider(100, min=20, max=20000, description="Frequency of wave 1", style=ws),
    fB=widgets.IntSlider(10000, min=20, max=20000, description="Frequency of wave 2", style=ws),
    noise=widgets.FloatSlider(1, min=0, max=4, description="Amplitude of noise", style=ws),
    K=widgets.IntSlider(5,1,10, description="Num hidden layers (of size 10)"),
    method=widgets.Select(options=['sgd', "adam", "lfbgs"], value='sgd',
                           description='Method for weight optimization:',
                           disabled=False))
def generate_sound(A: float, B:float, fA:float, fB:float, noise: float, method:str, K:int):
    T = 2
    pA, pB = 0, 0
    _, signal = generate_noisy_signal(sample_rate.value, np.array([A, B]), np.array([fA, fB]), np.array([pA, pB]),
                                      noise=noise, T=T)
    audio = Audio(signal, rate=sample_rate.value, autoplay=True)
    display(audio)
    target_labels = list(target_map.keys())
    
    hidden_layers = np.full(K, 2)
    print(labels)
    clf = MLPClassifier(solver=method, hidden_layer_sizes=hidden_layers, random_state=random_state,
                        activation="relu")
    clf.fit(X_train, y_train)

    predicted_label = clf.predict(X_test)
    print(predicted_label)


interactive(children=(FloatSlider(value=2.0, description='Amplitude of wave 1', max=4.0, style=SliderStyle(des…